<a href="https://colab.research.google.com/github/rdrgzbhnng/marketplace_ai/blob/main/marketplace_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Instalando o SDK do Google
!pip install -q -U google-generativeai

#Importações e configurações iniciais
import json
import requests
import numpy as np
import pandas as pd
import google.generativeai as genai

from google.colab import userdata

API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=API_KEY)

for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [15]:
# Importar o JSON
json_url = 'https://raw.githubusercontent.com/rdrgzbhnng/marketplace_ai/main/liquidity.json'
response = requests.get(json_url)

if response.status_code == 200:
    # Request successful, proceed to parse JSON data
    json_data = response.json()
else:
    # Handle error (e.g., invalid URL or website down)
    print('Error fetching JSON data:', response.status_code)

# Convertirlo em um DataFrame
liquidity_data = pd.DataFrame(json_data)

# Explorar data
liquidity_data.head()

,ID,From,To,Distance,Duration,Date,Type of truck,Weight,Length,Contact,Equipament,Cerificates,Pallets
0,A00001,ES-28001 Madrid,ES-01001 Vitoria-Gasteiz,354 km,4h 58m,09/05,Box,"0,1 t",2 m,+34 756 56 45,Tail Lift,,0
1,N00033,ES-28864 Ajalvir,ES-08820 Prat De Llobregat,592 km,8h 1m,09/05,Tautliner,"0,1 t",1 m,+34 741 82 46,,,0
2,M00056,ES-08001 Barcelona,ES-24420 Fabero,927 km,12h 56m,09/05,Tautliner,"0,1 t",7 m,+34 708 38 36,,ADR,1
3,J00024,ES-28830 San Fernando De Henares,ES-20303 Irun,465 km,6h 26m,10/05,Tautliner,"0,2 t","0,4 m",+34 768 66 86,,,0
4,X00053,ES-29300 Archidona,ES-41440 Lora del Río,178 km,2h 53m,09/05,Frigo,"0,4 t",1 m,+34 764 56 88,,,1


In [29]:
#Embeddando
model = "models/embedding-001"

def embed_fn(title, text):
  return genai.embed_content(model=model,
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]


#Testando com o primeiro item
lq = liquidity_data.loc[0]
lg_title = lq['ID']
lg_text = "Oferta de carga com cargamento em " + lq['From'] + " e descarga em " + lq['To'] + ". Dia de carga: " + lq['Date'] + ". Distancia: " + lq['Distance'] + " (" + lq['Duration'] + "). Tipo de caminhão: " + lq['Type of truck'] + ". Contato: " + lq['Contact']
print(lg_text)

e = embed_fn(lg_title, lg_text)
print(e)

Oferta de carga com cargamento em ES-28001 Madrid e descarga em ES-01001 Vitoria-Gasteiz. Dia de carga: 09/05. Distancia: 354 km (4h 58m). Tipo de caminhão: Box. Contato: +34 756 56 45
[0.011551606, -0.015252097, -0.063652284, -0.031221857, 0.008007622, 0.02853825, 0.026214756, -0.0028536143, 0.023018489, 0.055049255, -0.033096775, -0.009131664, -0.03877212, -0.0042337165, -0.011912827, -0.029309379, 0.010935043, -0.003967653, -0.030747496, 0.009002927, 0.032180406, 0.017666241, -0.042544287, 0.0041577388, -0.030500498, -0.037822705, 0.007386213, -0.05106396, 0.010261339, 0.035573926, -0.07330719, 0.032481488, -0.076356895, 0.009261586, 0.0041878517, -0.065995835, 0.002555567, 0.026794374, -0.025292844, 0.028276699, 0.022129664, -0.022563117, -0.012614708, -0.03322369, 0.0065366197, 0.02257788, 0.022993872, -0.04479272, 0.034331784, -0.069092095, 0.01170213, -0.029873677, 0.049767, -0.0028419485, -0.0071618147, -0.043668117, 0.06239775, 0.00020015615, -0.011934183, -0.004089142, 0.0323

In [35]:
#Embeddando geral

liquidity_data["Embeddings"] = liquidity_data.apply(lambda row: embed_fn(str(row["ID"]), "Oferta de carga com cargamento em " + row['From'] + " e descarga em " + row['To'] + ". Dia de carga: " + row['Date'] + ". Distancia: " + row['Distance'] + " (" + row['Duration'] + "). Tipo de caminhão: " + row['Type of truck'] + ". Contato: " + row['Contact']), axis=1)

#Explorar data
liquidity_data.head()

,ID,From,To,Distance,Duration,Date,Type of truck,Weight,Length,Contact,Equipament,Cerificates,Pallets,Embeddings
0,A00001,ES-28001 Madrid,ES-01001 Vitoria-Gasteiz,354 km,4h 58m,09/05,Box,"0,1 t",2 m,+34 756 56 45,Tail Lift,,0,"[0.011551606, -0.015252097, -0.063652284, -0.0..."
1,N00033,ES-28864 Ajalvir,ES-08820 Prat De Llobregat,592 km,8h 1m,09/05,Tautliner,"0,1 t",1 m,+34 741 82 46,,,0,"[-0.0030881984, -0.01936838, -0.06700688, -0.0..."
2,M00056,ES-08001 Barcelona,ES-24420 Fabero,927 km,12h 56m,09/05,Tautliner,"0,1 t",7 m,+34 708 38 36,,ADR,1,"[0.016918762, -0.035643224, -0.07575403, -0.02..."
3,J00024,ES-28830 San Fernando De Henares,ES-20303 Irun,465 km,6h 26m,10/05,Tautliner,"0,2 t","0,4 m",+34 768 66 86,,,0,"[0.0023912638, -0.0123199895, -0.082529396, -0..."
4,X00053,ES-29300 Archidona,ES-41440 Lora del Río,178 km,2h 53m,09/05,Frigo,"0,4 t",1 m,+34 764 56 88,,,1,"[0.040530294, 0.00909406, -0.10161001, -0.0033..."


In [24]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(liquidity_data["Embeddings"]), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)
  return liquidity_data.iloc[indice]["ID"]

In [52]:
consulta = "Recomende uma carga com cargamento em ES-28001 Madrid"
resposta = gerar_e_buscar_consulta(consulta, liquidity_data, model)

print(f"Para esta direção de origem, tem um cargamento com o offer_id: {resposta}")

Para esta direção de origem, tem um cargamento com o offer_id: A00001
